In [12]:
import pandas as pd
import re


In [13]:
def print_full(df):
    pd.set_option('display.max_colwidth', 2000)
    # pd.set_option('display.width', 2000)

    print(df)
    # pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 2000)
# pd.set_option('display.float_format', '{:20,.2f}'.format)
# pd.set_option('display.max_colwidth', None)
# print(x)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.width')
# pd.reset_option('display.float_format')
# pd.reset_option('display.max_colwidth')

In [14]:
df = pd.read_csv("contrib_from_csv.csv", sep = ",", encoding = "utf-8", dtype= str)

if df.isnull().values.any() :
    print("y a t il des nan? ->",df.isnull().values.any())
    print("combien y a t il de nan? ->",df.isnull().values.sum())
    print("où sont les null? ->\n",df.isnull().sum())
    df[df.isnull().T.any()]
else :
    print("il n'y a pas de NaN")

il n'y a pas de NaN


In [15]:
# si besoin de compter le nombre d'occurence d'un pattern de regex
def count_occurrences(regex, text):
    pattern = re.compile(regex)
    matches = re.findall(pattern, text)
    return len(matches)

In [16]:
# tester les types du dataframe et vérifier que les cellules ne contiennent pas des mélanges de types
print(df.dtypes.nunique()>1)

df._is_mixed_type

for column in df.columns:
    print(pd.api.types.infer_dtype(df[column]))

False
string
string
string
string
string
string
string
string
string
string


In [17]:

def remove_dots_and_spaces(text):
    """
    détecte les acronymes et supprime les espaces et les points entre les lettres des acronymes

    Input : une string
    Output : la string avec les acronymes corrigés
    """
    
    pattern = r"([A-Z]((\.|\s)+)([A-Z]((\.|\s)+))+)\s"
    matches = re.findall(pattern, text)
    
    for match in matches:
        original_match = match[0]
        corrected_match = re.sub("\.|\s", "", original_match)
        text = text.replace(original_match, corrected_match+" ")
    
    return text



def acronym(doc_to_clean, batch_size) : 
    """
    applique remove_dots_and_spaces() sur chaque cellule de texte du dataframe

    Input : une colonne de textes sélectionnée dans un dataframe , une taille de batch
    Output : un dataframe avec toutes les string modifiées 
    """

    # count the number of batches
    len_data = doc_to_clean.shape[0]
    batch_total = int(len_data/batch_size)+ (len_data % batch_size > 0)

    # size of last bacth
    size_last_batch = len_data%batch_size
    first_idx = 0
    last_idx = 0

    output_df = pd.DataFrame()
    for batch in range(batch_total) :

        corrected_text = []
        #  detect last batch
        if batch == batch_total -1 :
            first_idx = last_idx
            last_idx += size_last_batch
        # 
        else :
            first_idx = last_idx
            last_idx += batch_size
        print(first_idx, last_idx)
        for text_cell in doc_to_clean[first_idx:last_idx] :
            # for truc in text_cell :
            corrected_text.append(remove_dots_and_spaces(text_cell))

        # concatene this batch list of text with the complete dataframe 
        output_df = pd.concat([output_df, pd.DataFrame({"acronym" : corrected_text})], ignore_index= True )
        del corrected_text

    return output_df

In [18]:
# # supprimer les illisibles de ce corpus (plein de types de codages différents)
kill_texte_illisible = lambda x : x.replace("texte illisible", " ")
kill_illisible_mot = lambda x : x.replace("[mot ]", " ")
soupir = lambda x : x.replace("[mot illisible]", " ")
soupir2 = lambda x : x.replace("[mots illisibles]", "")
kill_mot_illisble = lambda x : x.replace("mot illisible", " ")
kill_illisible = lambda x : x.replace("illisible", " ")
kill_illisible_plural = lambda x : x.replace("illisibles", " ")
kill_illisible_capslock = lambda x : x.replace("ILLISIBLE", " ")
kill_illisibleS_capslock = lambda x : x.replace("ILLISIBLES", " ")

df["clean"] = df["Contribution"].apply(kill_texte_illisible)
df["clean"] = df["clean"].apply(kill_illisible_mot)
df["clean"] = df["clean"].apply(soupir)
df["clean"] = df["clean"].apply(soupir2)
df["clean"] = df["clean"].apply(kill_mot_illisble)
df["clean"] = df["clean"].apply(kill_illisible)
df["clean"] = df["clean"].apply(kill_illisible_plural)
df["clean"] = df["clean"].apply(kill_illisible_capslock)
df["clean"] = df["clean"].apply(kill_illisibleS_capslock)

df["clean"] = df["clean"].replace("^il(?:l(?:(?:(?:isuble|sibil)|isubl)|sible)|lisible|ilsible|li?sble)$", " ", regex = True)


# application des fonctions de nettoyage 
# reconstruition des phrases
# en cas d'item suivi d'une majuscule, remplace par un point + majuscule, en cas de minuscule, espace + minuscule.
df["clean"] = df["clean"].replace({'\n\s?([A-Z])' : ".\\1"}, regex = True)
df["clean"] = df["clean"].replace({"\x0b\s?([A-Z])" : ".\\1"}, regex = True)
df["clean"] = df["clean"].replace({"x92\s?([A-Z])" : '.\\1'}, regex = True)
df["clean"] = df["clean"].replace({"\v\s?([A-Z])" : '.\\1'}, regex = True)

df["clean"] = df["clean"].replace({"\v\s?([a-z])" : ' \\1'}, regex = True)
df["clean"] = df["clean"].replace({"\n\s?([a-z])" : " \\1"}, regex = True)
df["clean"] = df["clean"].replace({"\x0b\s?([a-z])" : " \\1"}, regex = True)
df["clean"] = df["clean"].replace({"x92\s?([a-z])" : " \\1"}, regex = True)


# supprimer les fins de lignes restantes
join_last_breaklines = lambda x : x.replace("\n"," ") 
df["clean"] = df['clean'].apply(join_last_breaklines)

# supprimer les doubles ponctuations de fins de phrase, les doubles espaces
df["clean"] = df["clean"].replace({"[\.]{2,}" : "."}, regex = True)
df["clean"] = df["clean"].replace({"[?]{2,}" : "?"}, regex = True)
df["clean"] = df["clean"].replace({"[!]{2,}" : "!"}, regex = True)
df["clean"] = df["clean"].replace({"[:]{2,}" : ":"}, regex = True)
df["clean"] = df["clean"].replace({"[;]{2,}" : ";"}, regex = True)

df["clean"] = df["clean"].replace({"[\s]{2,}" : " "}, regex = True)


# nettoyage des mails
df["clean"] = df["clean"].replace('(\w\-?)+@(\w\-?)+[.]{1}[a-z]+', " ", regex = True)
# ((\w+(-?)|(\.?))+|(\w+)@(\w\-?)+[.]{1}[a-z]+)
# nettoyage des sites
df["clean"] = df["clean"].replace('(http:\/\/|https:\/\/|www.)((\w)+(\/?)(\w+))+(\.?)+((\w)+(\/?)(\w+))+', " ", regex = True)

# nettoyage des chiffres et dates? ça contient des informations aussi, 80km par ex.
# df["clean"] = df["clean"].replace('[0-9]+(\/?)[0-9]+(\/?)[0-9]+', "", regex = True)

# nettoyage des slashs
df["clean"] = df["clean"].replace('\\|\/', " ", regex = True)

# normalisation des acronymes
df["clean"] = acronym(df['clean'], batch_size=400)

# trace de mots récurrents sans contenus pertinents
mots_a_clean = ["fin page", "page écrire", "wanadoo", "vierge rayer", "page vierge", "text area", "gmail.com", "neuillyplaisance", "cordialement","bonjour", "au revoir", "<image>", ">", "<"]
for mot in mots_a_clean :
    that = lambda x : x.lower().replace(mot, " ")
    df["clean"] = df["clean"].apply(that)
df["clean"] = df["clean"].replace('■'," ")
# ■ n'a pas de code ASCII, cette fonction n'a probablement pas d'effet
df["clean"] = df["clean"].replace({"([i|I][s|S][f|F])" : " ISF "}, regex = True)



0 400
400 800
800 1200
1200 1600
1600 2000
2000 2400
2400 2800
2800 3200
3200 3600
3600 4000
4000 4400
4400 4800
4800 5200
5200 5600
5600 6000
6000 6400
6400 6800
6800 7200
7200 7600
7600 8000
8000 8400
8400 8800
8800 9200
9200 9600
9600 10000
10000 10400
10400 10800
10800 11200
11200 11600
11600 12000
12000 12400
12400 12800
12800 13200
13200 13600
13600 14000
14000 14400
14400 14800
14800 15200
15200 15600
15600 16000
16000 16400
16400 16800
16800 17200
17200 17600
17600 18000
18000 18400
18400 18800
18800 19200
19200 19600
19600 20000
20000 20400
20400 20800
20800 21200
21200 21600
21600 22000
22000 22400
22400 22800
22800 23200
23200 23600
23600 24000
24000 24400
24400 24800
24800 25200
25200 25600
25600 26000
26000 26400
26400 26800
26800 27200
27200 27600
27600 28000
28000 28400
28400 28800
28800 29200
29200 29600
29600 30000
30000 30400
30400 30800
30800 31200
31200 31600
31600 32000
32000 32400
32400 32800
32800 33200
33200 33600
33600 34000
34000 34400
34400 34800
34800 35200


In [19]:
df

,Catégorie,Date de réception,Code postal,Code INSEE,Numéro d'ordre arbitraire,Type Graphie TT,Numéro de page,Numéro séquentiel,Contribution,joined_id,clean
0,CC,190322,03350,03048,18236,MD,5,2,-Supprimer la retraite du Président de la Répu...,CC_03350_190322_03048_MD_18236,-supprimer la retraite du président de la répu...
1,CC,190322,07000,07184,18770,MD,3,1,Imprimé par SOUSA Victor - OH/CCM/GDN\nSujet :...,CC_07000_190322_07184_MD_18770,imprimé par sousa victor - oh ccm gdn.sujet : ...
2,CC,190322,07000,07184,18770,MD,6,2,-Indexation sur l'inflation\nLes salaires de t...,CC_07000_190322_07184_MD_18770,-indexation sur l'inflation.les salaires de to...
3,CC,190322,07000,07184,18770,MD,7,3,Respecter la parité homme/femme: alignement de...,CC_07000_190322_07184_MD_18770,respecter la parité homme femme: alignement de...
4,CC,190322,07000,07184,18770,MD,8,4,En annexe\nComment prendre le contrôle d'une N...,CC_07000_190322_07184_MD_18770,en annexe.comment prendre le contrôle d'une na...
...,...,...,...,...,...,...,...,...,...,...,...
225040,CC,190405,99999,99999,19346,MD,11,5,retraites sont insolvables ou ne retrouvent pa...,CC_99999_190405_99999_MD_19346,retraites sont insolvables ou ne retrouvent pa...
225041,CC,190405,99999,99999,19346,MD,13,6,FRANÇAIS DE L'ÉTRANGER...LES GRANDES INÉGALITÉ...,CC_99999_190405_99999_MD_19346,français de l'étranger.les grandes inégalités ...
225042,CC,190405,99999,99999,19346,MD,15,7,DEUXIÈME INÉGALITÉ\nAlors que la quasi totalit...,CC_99999_190405_99999_MD_19346,deuxième inégalité.alors que la quasi totalité...
225043,CC,190405,99999,99999,19346,MD,17,8,"Monsieur,\n* Trouver un accord avec l'AEFE par...",CC_99999_190405_99999_MD_19346,"monsieur, * trouver un accord avec l'aefe par ..."


In [20]:
# écriture du résultat dans un csv
df.to_csv("df_CC_cleaned.csv", index = False, sep = ",", encoding = "utf-8")


In [21]:
# tester l'intégrité du csv après to_csv()


# df = pd.read_csv("df_CC_cleaned.csv", sep = ",", encoding = "utf-8", dtype=str)

# for column in df.columns:
#     print(pd.api.types.infer_dtype(df[column]))


# if df.isnull().values.any() :
#     print("y a t il des nan? ->",df.isnull().values.any())
#     print("combien y a t il de nan? ->",df.isnull().values.sum())
#     print("où sont les null? ->\n",df.isnull().sum())
#     print(df[df.isnull().T.any()])
# else :
#     print("il n'y a pas de NaN")
    
# print(df[column][df[column].apply(lambda x: isinstance(x, type))])
# print(df._is_mixed_type)
# print(df.dtypes.nunique()>1)